In [21]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from numpy.linalg import norm
import pandas as pd
import numpy as np
import requests
import re
import multiprocessing
import plotly.graph_objects as go
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rayen\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [24]:
jd_data = pd.read_csv('data/jobs_data/JobsDataset.csv') 
jd_data.drop(['ID','Query'], axis=1, inplace=True)
jd_data['data'] = jd_data[['Job Title','Description']].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)
tagged_data = [TaggedDocument(words = word_tokenize(_d.lower()), tags = [str(i)]) for i, _d in enumerate(jd_data['data'])]
model = Doc2Vec(vector_size = 50,
min_count = 3,
epochs = 40,
alpha = 0.001,
workers = 4
)
model.build_vocab(tagged_data)

In [15]:
jd_data

,Job Title,Description,data
0,Junior Data Scientist Apprenticeship,Job Description As a Junior Data Scientist at ...,Junior Data Scientist Apprenticeship Job Descr...
1,"HBO Data Scientist, Content Science",OVERALL SUMMARY As a Data Scientist on the Dat...,"HBO Data Scientist, Content Science OVERALL SU..."
2,Junior Data Scientist,The Team: The Data science team is a newly for...,Junior Data Scientist The Team: The Data scie...
3,Jr Data Scientist,We now have a need for junior Data Scientist(s...,Jr Data Scientist We now have a need for junio...
4,"Data Scientist, Premium Content",Do you want to help guide the core business of...,"Data Scientist, Premium Content Do you want t..."
...,...,...,...
9993,Learning Architect,The opportunity Our customers understand that ...,Learning Architect The opportunity Our custome...
9994,Systems Architect,The NASA Ames Research Center has the requirem...,Systems Architect The NASA Ames Research Cente...
9995,Hardware Architect,What You'll Do Distinguished Engineer with pro...,Hardware Architect What You'll Do Distinguish...
9996,"Software Development Engineer, Full Stack – E...","Software Development Engineer, Full Stack – En...","Software Development Engineer, Full Stack – E..."


In [25]:
# Vocabulary building
model.build_vocab(tagged_data)
# Get the vocabulary keys
keys = model.wv.key_to_index.keys()
# Print the length of the vocabulary keys
print(len(keys))

27351


In [23]:
num_workers = multiprocessing.cpu_count()  # Use all available cores (4 in your case)
num_workers

16

In [26]:
# Train the model
for epoch in range(model.epochs):
    print(f"Training epoch {epoch+1}/{model.epochs}")
    model.train(tagged_data, 
                total_examples=model.corpus_count, 
                epochs=model.epochs)

model.save('cv_job_maching.model')
print("Model saved")

Training epoch 1/40
Training epoch 2/40
Training epoch 3/40
Training epoch 4/40
Training epoch 5/40
Training epoch 6/40
Training epoch 7/40
Training epoch 8/40
Training epoch 9/40
Training epoch 10/40
Training epoch 11/40
Training epoch 12/40
Training epoch 13/40
Training epoch 14/40
Training epoch 15/40
Training epoch 16/40
Training epoch 17/40
Training epoch 18/40
Training epoch 19/40
Training epoch 20/40
Training epoch 21/40
Training epoch 22/40
Training epoch 23/40
Training epoch 24/40
Training epoch 25/40
Training epoch 26/40
Training epoch 27/40
Training epoch 28/40
Training epoch 29/40
Training epoch 30/40
Training epoch 31/40
Training epoch 32/40
Training epoch 33/40
Training epoch 34/40
Training epoch 35/40
Training epoch 36/40
Training epoch 37/40
Training epoch 38/40
Training epoch 39/40
Training epoch 40/40
Model saved


In [28]:
def preprocess_text(text):
    # Convert the text to lowercase
    text = text.lower()
    
    # Remove punctuation from the text
    text = re.sub('[^a-z]', ' ', text)
    
    # Remove numerical values from the text
    text = re.sub(r'\d+', '', text)
    
    # Remove extra whitespaces
    text = ' '.join(text.split())
    
    return 
def remove_accents(input_string):
    accents = {'é': 'e', 'è': 'e', 'ê': 'e', 'ë': 'e', 'É': 'E', 'È': 'E', 'Ê': 'E', 'Ë': 'E',
        'à': 'a',
        'â': 'a',
        'ç': 'c',
        'î': 'i',
        'ï': 'i',
        'ô': 'o',
        'ù': 'u',
        'û': 'u'
        # Add more replacements as needed
    }
def remove_initial_newlines(text):
    sentence_found = False
    new_text = ''

    for char in text:
        if not sentence_found:
            if char == '\n':
                continue
            else:
                sentence_found = True
                new_text += char
        else:
            new_text += char

    return new_text
def remove_first_lines(text):
    first_lines_removed= remove_initial_newlines(text)
    return '\n'.join(first_lines_removed.split('\n', 1)[1:])

In [ ]:
import sys
sys.path.append('..')
from tika import parser
def read_file(file,path):
  """Reads a file and returns the content."""
  try:
    parsed_pdf = parser.from_file(path+'/'+file)
    non_accent = remove_accents(parsed_pdf['content'].lower())
    return remove_first_lines(non_accent) 
  except Exception as e:
    # Catch any other exceptions and log error message
    logging.error("Error while reading the file: " + str(e))
    return None


In [ ]:
# Apply to CV and JD

input_CV = preprocess_text(resume)
input_JD = preprocess_text(jd)